<a href="https://colab.research.google.com/github/Syamchand123/pytorch_handson/blob/main/pytorch_handson_2(dataloader%2Creal_ANN%2CGPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:
import os
os.path.getsize('/content/fashion-mnist_train.csv')

133047193

In [21]:
df=pd.read_csv('/content/fashion-mnist_train.csv')
df_test=pd.read_csv('/content/fashion-mnist_test.csv')

In [23]:
df.shape
df_test.shape

(10000, 785)

In [24]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
x_train=df.iloc[:,1:].values
y_train=df.iloc[:,0].values
x_train=x_train/255.0


x_test=df_test.iloc[:,1:].values
y_test=df_test.iloc[:,0].values
x_test=x_test/255.0

In [26]:
x_train_tensor=torch.from_numpy(x_train)
y_train_tensor=torch.from_numpy(y_train)

In [27]:
# dataset class

class mydataset(Dataset):

  def __init__(self,features,lables):

    self.features=torch.tensor(features,dtype=torch.float32)
    self.lables=torch.tensor(lables,dtype=torch.long)


  def __len__(self):

    return len(self.features)


  def __getitem__(self,index):

    return self.features[index],self.lables[index]



In [28]:
training_dataset=mydataset(x_train,y_train)
test_dataset=mydataset(x_test,y_test)

In [29]:
train_loader=DataLoader(training_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)

In [30]:
# defining neural network

class myneuralnet(nn.Module):

  def __init__(self,num_of_features):

    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_of_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )


  def forward(self,features):

    out=self.model(features)
    return out

In [55]:
#set parameters
learning_rate=0.001
epochs=10

In [49]:
#instance model ,loss funtion,optimizers

model=myneuralnet(x_train.shape[1])
model= model.to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=nn.CrossEntropyLoss()



In [56]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features,batch_lables in train_loader:


    batch_features=batch_features.to(device)
    batch_lables=batch_lables.to(device)
    #forward pass
    y_pred=model(batch_features)


    #loss calculation

    loss=criterion(y_pred,batch_lables)

    # zero grad
    optimizer.zero_grad()

    #backward propagation

    loss.backward()

    #update weights

    optimizer.step()

    total_epoch_loss += loss.item()

  avg_loss=total_epoch_loss/len(train_loader)

  print(f'Epoche :{epoch+1}, Loss :{avg_loss}')




Epoche :1, Loss :0.15121618028023223
Epoche :2, Loss :0.15006648474680717
Epoche :3, Loss :0.1445341535917382
Epoche :4, Loss :0.14326889912631594
Epoche :5, Loss :0.13787080277937022
Epoche :6, Loss :0.13396347227540098
Epoche :7, Loss :0.13269389777986415
Epoche :8, Loss :0.12809924693452493
Epoche :9, Loss :0.1274628532555963
Epoche :10, Loss :0.12326195795756223


In [57]:
#evaluation
model.eval()



myneuralnet(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [58]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)



0.8945
